In [1]:
pip install yfinance

     |████████████████████████████████| 6.4 MB 7.3 MB/s 
     |████████████████████████████████| 63 kB 654 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [25]:
pip install psaw

In [51]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import time
label_encoder = preprocessing.LabelEncoder()
import datetime
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date, timedelta
from pandas import DataFrame
import pandas as pd
from bs4.element import DEFAULT_OUTPUT_ENCODING


import yfinance as yf

**Option version (default must be on)**

**Turning it into a function to compare predictions (scroll down)**

In [ ]:
days_back = 50
print('enter the # of categories you would like to analyze (3 MAX)')
num_catzz = int(input())
## of days per return period
xyy = {'title':[]}
print('enter time interval (## days) that you want to train your model on')
print('note: retrain every time you change this in the testing (analysis) cell')
shift = int(input())
for x in range(num_catzz):
  print('enter category #'+str(x))
  xyy['title'].append(str(input()).lower())
xyy = pd.DataFrame(xyy)
urls = [
  'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
  'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
  'https://companiesmarketcap.com/airlines/largest-airlines-by-market-cap/',
  'https://companiesmarketcap.com/airports/largest-airport-operating-companies-by-market-cap/',
  'https://companiesmarketcap.com/aircraft-manufacturers/largest-aircraft-manufacturers-by-market-cap/',
  'https://companiesmarketcap.com/banks/largest-banks-by-market-cap/',
  'https://companiesmarketcap.com/hotels/largest-hotel-companies-by-market-cap/',
  'https://companiesmarketcap.com/pharmaceuticals/largest-pharmaceutical-companies-by-market-cap/',
  'https://companiesmarketcap.com/e-commerce/largest-e-commerce-companies-by-market-cap/',
  'https://companiesmarketcap.com/healthcare/largest-healthcare-companies-by-market-cap/',
  'https://companiesmarketcap.com/ports/largest-port-operating-companies-by-market-cap/',
  'https://companiesmarketcap.com/professional-services/largest-professional-service-companies-by-market-cap/',
  'https://companiesmarketcap.com/food/largest-food-companies-by-market-cap/',
  'https://companiesmarketcap.com/restaurant-chains/largest-restaurant-chain-companies-by-market-cap/',
  'https://companiesmarketcap.com/software/largest-software-companies-by-market-cap/',
  'https://companiesmarketcap.com/semiconductors/largest-semiconductor-companies-by-market-cap/',
  'https://companiesmarketcap.com/tobacco/largest-tobacco-companies-by-market-cap/',
  'https://companiesmarketcap.com/financial-services/largest-financial-service-companies-by-market-cap/',
  'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/',
  'https://companiesmarketcap.com/electricity/largest-electricity-companies-by-market-cap/',
  'https://companiesmarketcap.com/delivery-services/largest-delivery-companies-by-market-cap/',
  'https://companiesmarketcap.com/media-press/largest-media-and-press-companies-by-market-cap/',
  'https://companiesmarketcap.com/alcoholic-beverages/largest-alcoholic-beverage-companies-by-market-cap/',
  'https://companiesmarketcap.com/beverages/largest-beverage-companies-by-market-cap/',
  'https://companiesmarketcap.com/clothing/largest-clothing-companies-by-market-cap/',
  'https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap/',
  'https://companiesmarketcap.com/railways/largest-railways-companies-by-market-cap/',
  'https://companiesmarketcap.com/insurance/largest-insurance-companies-by-market-cap/',
  'https://companiesmarketcap.com/real-estate/largest-real-estate-companies-by-market-cap/',
  'https://companiesmarketcap.com/chemicals/largest-chemical-companies-by-market-cap/',
  'https://companiesmarketcap.com/investment/largest-investment-companies-by-market-cap/',
  'https://companiesmarketcap.com/telecommunication/largest-telecommunication-companies-by-market-cap/',
  'https://companiesmarketcap.com/retail/largest-retail-companies-by-market-cap/',
  'https://companiesmarketcap.com/internet/largest-internet-companies-by-market-cap/',
  'https://companiesmarketcap.com/construction/largest-construction-companies-by-market-cap/',
  'https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/'
]
a = []
for x in range(len(urls)):
    a.append(urls[x].split('/')[3])
print('enter the "anchor stock" for your algorithm... just pick a stock from one of your selected categories')
print('refer to companiesmarketcap.com to find stock options')
ticker = input().upper()    

import requests
import time
import bs4
from bs4 import BeautifulSoup
cats = ['automakers','airlines','aircraft-manufacturers','banks','pharmaceuticals','e-commerce','healthcare','ports','professional-services','food','restaurant-chains','software','semiconductors','tobacco','financial-services','electricity','courier-services','media/press','alcoholic-beverages','beverages','clothing','mining','railways','insurance','real-estate','chemicals','investment','telecomunication','retail','internet','construction','tech']
cat_names = {'category':[],'names':[],'ticker':[]}
try:
    for x in range(len(urls)):
        url = urls[x]
        html = requests.get(url).text
        soup = bs4.BeautifulSoup(html, "html.parser")
        for tag in soup.findAll("div", {"class": "name-div"}):
                cat_names['names'].append(tag.findNext("div").text)
                cat_names['category'].append(a[x])
        for tag in soup.findAll("div", {"class": "company-name"}):
                cat_names['ticker'].append(tag.findNext().text)

except:
    print(x)
else:
  print('no scrape')

  cat_names = pd.DataFrame(cat_names)
  print('enter your industry categories, according to the number you referrenced above')
  gg = []  
  tech = pd.DataFrame()
  cat_holder = []
  for x in range(num_catzz):
    x = xyy['title'][x]
    cat_holder.append(x)
    tech = cat_names['ticker'][cat_names['category']==x]
    gg.append(tech)
  tech = pd.concat(gg,ignore_index = True)
  tech = tech.drop_duplicates()
  tickk = {'ticker':['GME']}
  tickk = pd.DataFrame(tickk)
  tech.append(tickk['ticker'])
    

if 1<2:

  for x in range(len(tech)):
    try: #from yfinance module (searching daily returns from tech symbols on marketcap.com)
        xx  = yf.Ticker(tech.iloc[x])
        df = xx.history(period="max")
        df.columns.values[0] = tech.iloc[x]+" Open"
        df.columns.values[1] = tech.iloc[x]+" High"
        df.columns.values[2] = tech.iloc[x]+" Low"
        df.columns.values[3] = tech.iloc[x]+" Close"
        df.columns.values[4] = tech.iloc[x]+" Volume"
        df.columns.values[5] = tech.iloc[x]+" Dividends"
        df.columns.values[6] = tech.iloc[x]+" Stock Splits"
        tech.iloc[x] = df.reset_index()
    except:
      print(tech.iloc[x])
  msft = yf.Ticker(str(ticker).upper())
  df = msft.history(period="max")
  for ii in range(len(tech)):
    if len(tech.iloc[ii])>=days_back:
      df = pd.merge(df, tech.iloc[ii] ,on = 'Date')
    else:
      print('not long enough!')
  for x in range(num_catzz):
    x = xyy['title'][x]
    cat_holder.append(x)
    tech = (cat_names['ticker'][cat_names['category']==x])
    gg.append(tech)
  tech = pd.concat(gg,ignore_index = True)
  tech = tech.reset_index()
  tech = tech.drop(columns = 'index')
  tech = tech.drop_duplicates()
  tech.append(tickk['ticker'])

  tech = tech['ticker']
  aa = pd.DataFrame()
  vol = pd.DataFrame()
  ss = []
  for i in range(len(tech)):
    try:
      aa[str(tech.iloc[i])] = (df[str(tech.iloc[i])+' Close'].shift(-1*shift) - df[str(tech.iloc[i])+' Open'].shift(shift)).dropna()/df[str(tech.iloc[i])+' Open'].shift(shift).dropna()*100
      vol[str(tech.iloc[i])] = df[str(tech.iloc[i])+' Volume']
    except:
      ss.append(str(tech.iloc[i]))
  aa = round(aa)
  tech = pd.DataFrame(tech)
  dd = pd.DataFrame()
  rr = pd.DataFrame()
  for x in range(len(tech)):
    try:
      rr[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])
  qq = pd.DataFrame()
  for x in range(len(tech)):
    try:
      qq[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])
  ss = []
  df = pd.DataFrame()
  for x in range(len(tech)):
    try:
      df = qq.drop(columns = tech.iloc[x])
      ss.append(df)
    except:
      print(tech.iloc[x])
  pieces = []
  piecess = []
  for x in range(len(tech)):
    try:
      pieces.append(ss[x].iloc[-1*days_back:-1]) #for the training set
      piecess.append(ss[x].iloc[-1*days_back:]) #the test set (using the last x iteration to predict the next return)
    except:
      print(tech.iloc[x])
  df_final = pd.concat(pieces,axis = False, ignore_index = True)
  df_finall = pd.concat(piecess,axis= False, ignore_index = True )
else:
      print('no setup')
if 2>1:
  y = []
  for x in range(len(tech)):
    try:
      y.append(qq[tech['ticker'][x]].iloc[-1*days_back:].shift(shift*-1).dropna()) #shifting y back 1 to represent xx day(s) returns from xx day(s) returns 1 unit back
    except:
      print(tech.iloc[x])
  y = pd.concat(y, ignore_index = True)
  yy =y.reset_index()
  yy = yy.drop(columns = 'index')
  yy[0].loc[~(yy[0]==0)]
  yyy = pd.DataFrame()
  yy = yy.dropna()
  yyy['y'] = yy[0]
  yyy = yyy.reset_index()
  tt = round(df_final)
  ttt = round(df_finall)
  tt = tt.fillna(1)
  ttt = ttt.fillna(1)
  xx = tt.reset_index()

  xx = xx.drop(columns = 'index')
  xx = xx.reset_index()
  xxx = pd.merge(yyy,xx, on = 'index')
  y = xxx['y']
  xxx = xxx.drop(columns = ['index','y'])
  def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
  xxx = clean_dataset(xxx).reset_index()
  yy = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index')
  yy = yy['y'].astype(int)
  xxx = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index').drop(columns = ['y','index','level_0']).astype(int)
  x = xxx
  xx = xxx
  xxx = xxx
  y = yy
  yy = yy
  yyy = yy
else:
  print('no setup')
if 2>1:
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  clf=RandomForestClassifier(n_estimators=1000)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print('random forest score = '+str(metrics.accuracy_score(y_test, y_pred)))
else:
  print('no random forest')
if 1>2:
  xx, yy = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

  model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
else:
  print('no logistic regression')
if 2>1:
  X_train, X_test, y_train, y_test = train_test_split(xxx, yyy, test_size=0.2)
  lr_list = [0.05]
  for learning_rate in lr_list:
      gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
      gb_clf.fit(X_train, y_train)
      print("Learning rate: ", learning_rate)
      print("Gradient Boosted Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
      print("Gradient Boosted Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))
else:
  print('no gradient boosted forests')
if 2>1:
  xgb_clf = XGBClassifier()
  xgb_clf.fit(X_train, y_train)
  score = xgb_clf.score(X_test, y_test)
  print('XGB Score = '+str(score))
  xt = x.head(days_back)
  yt = x.iloc[days_back:days_back][str(ticker).upper()]
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_xgb = pd.DataFrame()
  df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
  df_compare_xgb['actual'] = yt
  df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
  dd = len(df_compare_xgb[df_compare_xgb['dif']==0])
  accuracy = dd/len(df_compare_xgb)*100
  print('xgb forest accuracy for last '+str(shift)+' '+'AAPL_test'+' '+str(accuracy))
else:
  print('no xg boosted forest')

enter the # of categories you would like to analyze (3 MAX)
3
enter time interval (## days) that you want to train your model on
note: retrain every time you change this in the testing (analysis) cell
4
enter category #0
all-assets-by-market-cap
enter category #1
tech
enter category #2
banks
enter the "anchor stock" for your algorithm... just pick a stock from one of your selected categories
refer to companiesmarketcap.com to find stock options
gme
no scrape
enter your industry categories, according to the number you referrenced above
random forest score = 0.03142857142857143
no logistic regression
Learning rate:  0.05
Gradient Boosted Accuracy score (training): 0.078
Gradient Boosted Accuracy score (validation): 0.044


In [91]:
bticker

'GME'

,ticker
0,GME


0         AAPL
1         MSFT
2         GOOG
3         AMZN
4         TSLA
        ...   
196      SWDBF
197    1050.SR
198        CFG
199    2884.TW
0          GME
Name: ticker, Length: 201, dtype: object

In [69]:
ticker['title']

'0    GME\nName: title, dtype: object'

In [46]:
def option_pricing(ttt,yt,ticker,num_categories ,default, days_back,descriptive,scrape,num_trees,setup,shift_change,current_price,div,r,shift):
  #------------------------------------------------------

  #testing + graphing
  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_rf = pd.DataFrame()
  df_compare_rf['prediction'] = list(clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_rf.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  yt
  df_compare_rf['actual'] = yt
  df_compare_rf['dif'] = abs(df_compare_rf['actual']-df_compare_rf['prediction'])
  dd = len(df_compare_rf[df_compare_rf['dif']==0])
  accuracy = dd/len(df_compare_rf)*100
  print('random forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  dd = df['Open'].iloc[int(len(df))-days_back-1:]
  df_ = {'open':[]}
  for x in range(len(dd)):
    df_['open'].append(int(dd[x]))
  df_final = pd.DataFrame()
  df_final[str(ticker.upper())+' prediction'] = df_['open'] + df_compare_rf['prediction']/100*df_['open']
  y1 = df_final[str(ticker.upper())+' prediction']


  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_xgb = pd.DataFrame()
  df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_xgb.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  df_compare_xgb['actual'] = yt
  df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
  ddd = len(df_compare_xgb[df_compare_xgb['dif']==0])
  accuracyy = ddd/len(df_compare_xgb)*100
  print('xgb forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracyy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  ddd = df['Open'].iloc[int(len(df))-days_back-1:]
  df__ = {'open':[]}
  for x in range(len(ddd)):
    df__['open'].append(int(ddd[x]))
  df_final_xgb = pd.DataFrame()
  df_final_xgb[str(ticker.upper())+' prediction'] = df__['open'] + df_compare_xgb['prediction']/100*df__['open']
  x11 = df_final_xgb[str(ticker.upper())+' prediction']


  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_gb = pd.DataFrame()
  df_compare_gb['prediction'] = list(gb_clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_gb.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  df_compare_gb['actual'] = yt
  df_compare_gb['dif'] = abs(df_compare_gb['actual']-df_compare_gb['prediction'])
  dddd = len(df_compare_gb[df_compare_gb['dif']==0])
  accuracyyy = dddd/len(df_compare_gb)*100
  print('gb forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracyyy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  dddd = df['Open'].iloc[int(len(df))-days_back-1:]
  df___ = {'open':[]}
  for x in range(len(dddd)):
    df___['open'].append(int(dddd[x]))
  df_finalll = pd.DataFrame()
  df_finalll[str(ticker.upper())+' prediction'] = df___['open'] + df_compare_gb['prediction']/100*df___['open']
  x111 = df_finalll[str(ticker.upper())+' prediction']

  df_check = pd.DataFrame()
  df_check['rf'] = y1
  df_check['gb'] = x111
  df_check['xgb'] = x11
  df_check['actual'] = dd.reset_index().drop(columns = ['Date']).head(days_back)

  plt.figure(figsize = (20, 10))
  ax = y1.head(days_back).plot(label='rf prediction')
  ax = (df_['open'] + df_compare_rf['actual']/100*df_['open']).plot(label='Actual')
  ax = x11.head(days_back).plot(label='xgb prediction')
  ax = x111.head(days_back).plot(label='gb prediction')
  ax.set_xlabel('Date')
  ax.set_ylabel(str(ticker.upper()))
  plt.axvline(x = days_back-1, color = 'b', label = 'axvline - full height')

  leg = plt.legend()
  for line in leg.get_lines():
      line.set_linewidth(10)
  for text in leg.get_texts():
      text.set_fontsize('x-large')

  plt.legend()
  plt.show()
  plt.savefig('predictions plot for '+str(ticker.upper())+'.png')

  print(df_check)

  def options_chain(symbol):
      msft = yf.Ticker(symbol)
      exps = msft.options
      ops = pd.DataFrame()
      for e in exps:
          opt = msft.option_chain(e)
          opt = pd.DataFrame().append(opt.calls).append(opt.puts)
          opt['expirationDate'] = e
          ops = ops.append(opt, ignore_index=True)
      ops['expirationDate'] = pd.to_datetime(ops['expirationDate']) + timedelta(days = 1)
      ops['dte'] = (ops['expirationDate'] - datetime.today()).dt.days
      ops['CALL'] = ops['contractSymbol'].str[0:].apply(
          lambda x: "C" in x)
      ops[['bid', 'ask', 'strike']] = ops[['bid', 'ask', 'strike']].apply(pd.to_numeric)
      ops['mark'] = (ops['bid'] + ops['ask']) / 2*1000
      return ops

  df = options_chain(str(ticker.upper())).sort_values('dte',ascending=True)

  calls = df[df['CALL']==True]
  puts = df[df['CALL']==False]
  puts = puts.sort_values('expirationDate',ascending = True)
  calls = calls.sort_values('expirationDate',ascending = True)

  #black scholes formula
  def d1(S,K,T,r,div,sigma):
      return(log(S/K)+(r-div+sigma**2/2.)*T)/(sigma*sqrt(T))
  def d2(S,K,T,r,div,sigma):
      return d1(S,K,T,r,div,sigma)-sigma*sqrt(T)
  def bs_call(S,K,T,r,div,sigma):
      return S*norm.cdf(d1(S,K,T,r,div,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,div,sigma))
  def bs_put(S,K,T,r,div,sigma):
      return K*exp(-r*T)-S+bs_call(S,K,T,r,div,sigma)
  print('puts:')
  S = current_price
  S1 =  float((df_['open'] + df_compare_rf['actual']/100*df_['open']).drop(columns = 'index')[days_back])
  difference = S1 - S
  pdifference = difference/S

  try:
    Tp = float(puts['dte'].reset_index().drop(columns = 'index')['dte'][xxe])/365
    Kp = float(puts['strike'].reset_index().drop(columns = 'index')['strike'][xxe])
    sigmap = float(puts['impliedVolatility'].reset_index().drop(columns = 'index')['impliedVolatility'][xxe])
    
    #FORMULA: C = price*N( [ln(price/strike)+interest_rate-divYield+vol^2/2)*T]/(vol*sqrt(T)) ] - strike*e^(interest rate * time)*N(d1 - vol*sqrt(T))
    print('Black Scholes Results...')
    zvvv = str(float((puts['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+puts['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('Price of put: '+zvvv)
    zvv = str(Kp-current_price-float((puts['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+puts['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('"immdiate excercise value" of put:'+zvv)
    zv = str(bs_put(S,Kp,Tp,r,div,sigmap))
    print('Black-Scholes expected value of put(from stock open today): '+zv)
    zzv = str(Tp)[:4]+' day(s): '+str(bs_put(S1,Kp,Tp,r,div,sigmap))
    print('Value of put according to avg of ensemble predictsions in '+zzv)
    print(' ---------- ')
  except:
    print('no puts for '+str(ticker.upper()))
  print(' ---------- ')
  print('calls:')
  try:
    Tc = float(calls['dte'].reset_index().drop(columns = 'index')['dte'][xxe])/365
    Kc = float(calls['strike'].reset_index().drop(columns = 'index')['strike'][xxe])
    sigmac = float(calls['impliedVolatility'].reset_index().drop(columns = 'index')['impliedVolatility'][xxe])
    cvvv = str(float((calls['bid'].reset_index().drop(columns = 'index')['bid'][x])+calls['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('Price of call: '+cvvv)
    cvv = str(current_price-Kc-float((calls['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+calls['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('"immdiate excercise value" of call:'+cvv)
    cv = str(bs_call(S,Kc,Tc,r,div,sigmac))
    print('Black-Scholes expected value of call(from stock open today): '+cv)
    ccv = str(Tc)[:]+' day(s): '+str(bs_call(S1,Kc,Tc,r,div,sigmac))
    print('Value of call according to avg of ensemble predictsions in '+ccv)
  except:
    print('no calls for '+str(ticker.upper()))
  
  return ticker, cvvv, cvv, cv,ccv, zvvv, zvv, zv,zzv, S1, difference, pdifference
       # ticker, price of call, immediate excercise value call, BS value, BS pred value, same for put, PRED_PRICE, PRED DIF, PRED P%
xcer =0


**Running Multiple Predictions...**

In [50]:
#default
num_categories = 3
xxe = 0
shift = xxe + 4
r = 0.02353
div = 0
days_back = 50
num_trees = 1000
start_date = '2021-09-03'
descriptive = 'yes'
cat_change = 'afdsa'

print('shift is set to 4 --> 4 day returns (go to very top of this cell and change if you want to predict sooner / farhther out (option pricing only works for 4 right now)')

if cat_change == 'faasd':
  #if you want to change cats
  scrape = 'yes'
  num_trees = 1000
  setup = 'on'
  default = 'on'
  shift_change = 'on'
  random_forest = 'yes'
  xgb = 'yes'
  logi_regression = 'no'
  gradient_boosted = 'yes'
  print('not trained')
else:
  #already trained model on cats
  num_trees = 1000
  scrape = 'no'
  setup = 'off'
  default = 'on'
  shift_change = 'off'
  random_forest = 'off'
  xgb = 'off'
  logi_regression = 'off'
  gradient_boosted = 'off'
  print('already trained')

  xcx = 'afdsa'
  xyy = xyy

  select_group = "group"


  if select_group =='single':
    print('enter the # of stocks you would like to analyze')
    num_preds = int(input())
    current_price = {'title':[]}
    ticker = {'title':[]}
    ticker = pd.DataFrame(ticker)
    current_price = pd.DataFrame(current_price)

    for x in range(num_preds):
      print('enter stock symbol for the '+str(x)+'th stock that you would like to analyze')
      ticker['title'].append(str(input().upper()))
      print('enter the price for your '+str(x)+'th stock')
      current_price['title'].append(float(input()))
  else:
    print('>1 predictions')

  if select_group =='group':  
    print('enter the # of stocks you would like to analyze')
    num_preds = int(input())
    current_price = {'title':[]}
    ticker = {'title':[]}
    
    for x in range(num_preds):
      print('enter stock symbol for the '+str(x)+'th stock that you would like to analyze')
      ticker['title'].append(str(input().upper()))
      print('enter the price for your '+str(x)+'th stock')
      current_price['title'].append(float(input()))
    ticker = pd.DataFrame(ticker)
    current_price = pd.DataFrame(current_price)

  else:
    print('not a select group')

  gg_ = pd.DataFrame()
  for xz in range(num_preds):
      ttt = ttt
      gg_[str(ticker['title'][xz])] = option_pricing(ttt,yt,ticker['title'][xz],num_catzz ,default, days_back,descriptive,scrape,num_trees,setup,shift_change,current_price['title'][xz],div,r,shift)
  resultz = pd.DataFrame()
  resultz['ticker'] = gg_.iloc[0]
  resultz['callPrice'] = gg_.iloc[1]
  resultz['immediateExcerciseCallValue'] = gg_.iloc[2]
  resultz['callBScholes'] = gg_.iloc[3]
  resultz['callBScholesPred'] = gg_.iloc[4]
  resultz['putPrice'] = gg_.iloc[5]
  resultz['immediateExcercisePutValue'] = gg_.iloc[6]
  resultz['putBScholes'] = gg_.iloc[7]
  resultz['putBScholesPred'] = gg_.iloc[8]
  resultz['predictedPrice'] = gg_.iloc[9]
  resultz['predicted%Change'] = gg_.iloc[10]
 # price of call, immediate excercise value call, BS value, BS pred value, same for put, PRED_PRICE, PRED DIF, PRED P%
resultz.head(50)

shift is set to 4 --> 4 day returns (go to very top of this cell and change if you want to predict sooner / farhther out (option pricing only works for 4 right now)
already trained
>1 predictions
enter the # of stocks you would like to analyze
1
enter stock symbol for the 0th stock that you would like to analyze
gme
enter the price for your 0th stock
146.93


NameError: ignored

**Just prediction / chart**

In [ ]:
from bs4.element import DEFAULT_OUTPUT_ENCODING
print('default (on/off)')
print("       * specs: up to 3 categories, you choose ticker+forecast period, 2k trees, trained on last 50 days of each stock's returns!")
default = input().lower()
print('input ticker')
ticker = input().upper()
print('model will predict xx days into the future')
shift = int(input())

if default == 'on':
  num_categories = 3
  days_back = 50
  descriptive = 'yes'
  scrape = 'yes'
  num_trees = 2000
  setup = 'on'
  shift_change = 'on'
  start_date = '2021-09-03'
else:
  print('Enter the # of stock "categories" you would like to include')
  print('      *ensure that the ticker you want to predict is in this category')
  print('      *(find categories @ companiesmarketcap.com)')
  print()
  print('      *3 categories max! (bug)')
  num_categories = int(input())
  descriptive = 'bugged'
  print('the stocks in your selected categories will be stacked on top of each other for training purposes, how many days back would you like each stock to go? (i.e. 10 days: 10 days of past returns for each stock):')
  days_back = int(input())
  print('how many trees do you want the random forest to have?:')
  num_trees = int(input())
  print('Do you need to scrape companymarketcap.com (yes/no) (only need to do once / runtime):')
  scrape = input()
  print('For each stock, we are predicting returns xx days into the future. How many days would you like to lag your data:')
  shift = int(input())
  start_date = '2021-09-03'
  print('yahoo finance data collection (on/off) (every time you change category selection):')
  setup = input()
  print('Quick setup (on/off):')
  shift_change = input()

print('run a random forest: (yes/no)')
random_forest = input()
print('run a xgboost (yes/no):')
xgb = input()
print('run a logit (yes/no):')
logi_regression = input()
print('run a gradient boosted (yes/no):')
gradient_boosted = input()

#------------------------------------------------------

if scrape == 'yes':
  urls = [
    'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
    'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
    'https://companiesmarketcap.com/airlines/largest-airlines-by-market-cap/',
    'https://companiesmarketcap.com/airports/largest-airport-operating-companies-by-market-cap/',
    'https://companiesmarketcap.com/aircraft-manufacturers/largest-aircraft-manufacturers-by-market-cap/',
    'https://companiesmarketcap.com/banks/largest-banks-by-market-cap/',
    'https://companiesmarketcap.com/hotels/largest-hotel-companies-by-market-cap/',
    'https://companiesmarketcap.com/pharmaceuticals/largest-pharmaceutical-companies-by-market-cap/',
    'https://companiesmarketcap.com/e-commerce/largest-e-commerce-companies-by-market-cap/',
    'https://companiesmarketcap.com/healthcare/largest-healthcare-companies-by-market-cap/',
    'https://companiesmarketcap.com/ports/largest-port-operating-companies-by-market-cap/',
    'https://companiesmarketcap.com/professional-services/largest-professional-service-companies-by-market-cap/',
    'https://companiesmarketcap.com/food/largest-food-companies-by-market-cap/',
    'https://companiesmarketcap.com/restaurant-chains/largest-restaurant-chain-companies-by-market-cap/',
    'https://companiesmarketcap.com/software/largest-software-companies-by-market-cap/',
    'https://companiesmarketcap.com/semiconductors/largest-semiconductor-companies-by-market-cap/',
    'https://companiesmarketcap.com/tobacco/largest-tobacco-companies-by-market-cap/',
    'https://companiesmarketcap.com/financial-services/largest-financial-service-companies-by-market-cap/',
    'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/',
    'https://companiesmarketcap.com/electricity/largest-electricity-companies-by-market-cap/',
    'https://companiesmarketcap.com/delivery-services/largest-delivery-companies-by-market-cap/',
    'https://companiesmarketcap.com/media-press/largest-media-and-press-companies-by-market-cap/',
    'https://companiesmarketcap.com/alcoholic-beverages/largest-alcoholic-beverage-companies-by-market-cap/',
    'https://companiesmarketcap.com/beverages/largest-beverage-companies-by-market-cap/',
    'https://companiesmarketcap.com/clothing/largest-clothing-companies-by-market-cap/',
    'https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap/',
    'https://companiesmarketcap.com/railways/largest-railways-companies-by-market-cap/',
    'https://companiesmarketcap.com/insurance/largest-insurance-companies-by-market-cap/',
    'https://companiesmarketcap.com/real-estate/largest-real-estate-companies-by-market-cap/',
    'https://companiesmarketcap.com/chemicals/largest-chemical-companies-by-market-cap/',
    'https://companiesmarketcap.com/investment/largest-investment-companies-by-market-cap/',
    'https://companiesmarketcap.com/telecommunication/largest-telecommunication-companies-by-market-cap/',
    'https://companiesmarketcap.com/retail/largest-retail-companies-by-market-cap/',
    'https://companiesmarketcap.com/internet/largest-internet-companies-by-market-cap/',
    'https://companiesmarketcap.com/construction/largest-construction-companies-by-market-cap/',
    'https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/'
  ]
  a = []
  for x in range(len(urls)):
      a.append(urls[x].split('/')[3])

  import requests
  import time
  import bs4
  from bs4 import BeautifulSoup
  cats = ['automakers','airlines','aircraft-manufacturers','banks','pharmaceuticals','e-commerce','healthcare','ports','professional-services','food','restaurant-chains','software','semiconductors','tobacco','financial-services','electricity','courier-services','media/press','alcoholic-beverages','beverages','clothing','mining','railways','insurance','real-estate','chemicals','investment','telecomunication','retail','internet','construction','tech']
  cat_names = {'category':[],'names':[],'ticker':[]}
  try:
      for x in range(len(urls)):
          url = urls[x]
          html = requests.get(url).text
          soup = bs4.BeautifulSoup(html, "html.parser")
          for tag in soup.findAll("div", {"class": "name-div"}):
                  cat_names['names'].append(tag.findNext("div").text)
                  cat_names['category'].append(a[x])
          for tag in soup.findAll("div", {"class": "company-name"}):
                  cat_names['ticker'].append(tag.findNext().text)

  except:
      print(x)
else:
  print('no scrape')

if setup == 'on':
  cat_names = pd.DataFrame(cat_names)
  print('enter your industry categories, according to the number you referrenced above')
  gg = []  
  tech = pd.DataFrame()
  cat_holder = []
  for x in range(num_categories):
    x = input()
    cat_holder.append(x)
    tech = (cat_names['ticker'][cat_names['category']==x])
    gg.append(tech)
  tech = pd.concat(gg,ignore_index = True)
  tech = tech.drop_duplicates()
    

  for x in range(len(tech)):
    try:
        xx  = yf.Ticker(tech.iloc[x])
        df = xx.history(period="max")
        df.columns.values[0] = tech.iloc[x]+" Open"
        df.columns.values[1] = tech.iloc[x]+" High"
        df.columns.values[2] = tech.iloc[x]+" Low"
        df.columns.values[3] = tech.iloc[x]+" Close"
        df.columns.values[4] = tech.iloc[x]+" Volume"
        df.columns.values[5] = tech.iloc[x]+" Dividends"
        df.columns.values[6] = tech.iloc[x]+" Stock Splits"
        tech.iloc[x] = df.reset_index()
    except:
      print(tech.iloc[x])

  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  for ii in range(len(tech)):
    if len(tech.iloc[ii])>=days_back:
      df = pd.merge(df, tech.iloc[ii] ,on = 'Date')
    else:
      print('not long enough!')
  for x in range(num_categories):
    tech = (cat_names['ticker'][cat_names['category']==pd.DataFrame(cat_holder)[0][x]])
    gg.append(tech)
  tech = pd.concat(gg,ignore_index = True)
  tech = tech.reset_index()
  tech = tech.drop(columns = 'index')
  tech = tech.drop_duplicates()
  tech = tech['ticker']
  aa = pd.DataFrame()
  vol = pd.DataFrame()
  ss = []
  for i in range(len(tech)):
    try:
      aa[str(tech.iloc[i])] = (df[str(tech.iloc[i])+' Close'].shift(-1*shift) - df[str(tech.iloc[i])+' Open'].shift(shift)).dropna()/df[str(tech.iloc[i])+' Open'].shift(shift).dropna()*100
      vol[str(tech.iloc[i])] = df[str(tech.iloc[i])+' Volume']
    except:
      ss.append(str(tech.iloc[i]))
  aa = round(aa)
  tech = pd.DataFrame(tech)
  dd = pd.DataFrame()
  rr = pd.DataFrame()
  for x in range(len(tech)):
    try:
      rr[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])
  qq = pd.DataFrame()
  for x in range(len(tech)):
    try:
      qq[tech.iloc[x]] = aa[tech.iloc[x]]
    except:
      print(tech.iloc[x])
  ss = []
  df = pd.DataFrame()
  for x in range(len(tech)):
    try:
      df = qq.drop(columns = tech.iloc[x])
      ss.append(df)
    except:
      print(tech.iloc[x])
  pieces = []
  piecess = []
  for x in range(len(tech)):
    try:
      pieces.append(ss[x].iloc[-1*days_back:-1])
      piecess.append(ss[x].iloc[-1*days_back:])
    except:
      print(tech.iloc[x])
  df_final = pd.concat(pieces,axis = False, ignore_index = True)
  df_finall = pd.concat(pieces,axis= False, ignore_index = True )
else:
      print('no setup')
if shift_change == 'on':
  y = []
  for x in range(len(tech)):
    try:
      y.append(qq[tech['ticker'][x]].iloc[-1*days_back:].shift(shift*-1).dropna())
    except:
      print(tech.iloc[x])
  y = pd.concat(y, ignore_index = True)
  yy =y.reset_index()
  yy = yy.drop(columns = 'index')
  yy[0].loc[~(yy[0]==0)]
  yyy = pd.DataFrame()
  yy = yy.dropna()
  yyy['y'] = yy[0]
  yyy = yyy.reset_index()
  tt = round(df_final)
  ttt = round(df_finall)
  tt = tt.fillna(1)
  ttt = ttt.fillna(1)
  xx = tt.reset_index()

  xx = xx.drop(columns = 'index')
  xx = xx.reset_index()
  xxx = pd.merge(yyy,xx, on = 'index')
  y = xxx['y']
  xxx = xxx.drop(columns = ['index','y'])
  def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
  xxx = clean_dataset(xxx).reset_index()
  yy = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index')
  yy = yy['y'].astype(int)
  xxx = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index').drop(columns = ['y','index','level_0']).astype(int)
  x = xxx
  xx = xxx
  xxx = xxx
  y = yy
  yy = yy
  yyy = yy
else:
  print('no setup')
if random_forest == 'yes':
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  clf=RandomForestClassifier(n_estimators=num_trees)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print('random forest score = '+str(metrics.accuracy_score(y_test, y_pred)))
else:
  print('no random forest')
if logi_regression == 'yes':
  xx, yy = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

  model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
else:
  print('no logistic regression')
if gradient_boosted == 'yes':
  X_train, X_test, y_train, y_test = train_test_split(xxx, yyy, test_size=0.2)
  lr_list = [0.05, 0.075, 0.1]
  for learning_rate in lr_list:
      gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
      gb_clf.fit(X_train, y_train)
      print("Learning rate: ", learning_rate)
      print("Gradient Boosted Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
      print("Gradient Boosted Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))
else:
  print('no gradient boosted forests')
if xgb == 'yes':
  xgb_clf = XGBClassifier()
  xgb_clf.fit(X_train, y_train)
  score = xgb_clf.score(X_test, y_test)
  print('XGB Score = '+str(score))
  xt = x.head(days_back)
  yt = x.iloc[days_back:days_back][ticker.upper()]
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_xgb = pd.DataFrame()
  df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
  df_compare_xgb['actual'] = yt
  df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
  dd = len(df_compare_xgb[df_compare_xgb['dif']==0])
  accuracy = dd/len(df_compare_xgb)*100
  print('xgb forest accuracy for last '+str(shift)+' '+ticker+' '+str(accuracy))
else:
  print('no xg boosted forest')

#testing + graphing
xt = ttt.head(days_back+1)
yt = yt.reset_index().drop(columns = 'index')
df_compare_rf = pd.DataFrame()
df_compare_rf['prediction'] = list(clf.predict(xt))
yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_rf.mean()[-1:])
yt = yt.reset_index().drop(columns = 'index')[0]
yt
df_compare_rf['actual'] = yt
df_compare_rf['dif'] = abs(df_compare_rf['actual']-df_compare_rf['prediction'])
dd = len(df_compare_rf[df_compare_rf['dif']==0])
accuracy = dd/len(df_compare_rf)*100
print('random forest accuracy for last '+str(shift)+' '+ticker+' '+str(accuracy))
msft = yf.Ticker(ticker.upper())
df = msft.history(period="max")
dd = df['Open'].iloc[int(len(df))-days_back-1:]
df_ = {'open':[]}
for x in range(len(dd)):
  df_['open'].append(int(dd[x]))
df_final = pd.DataFrame()
df_final[str(ticker.upper())+' prediction'] = df_['open'] + df_compare_rf['prediction']/100*df_['open']
y1 = df_final[str(ticker.upper())+' prediction']


xt = ttt.head(days_back+1)
yt = yt.reset_index().drop(columns = 'index')
df_compare_xgb = pd.DataFrame()
df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_xgb.mean()[-1:])
yt = yt.reset_index().drop(columns = 'index')[0]
df_compare_xgb['actual'] = yt
df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
ddd = len(df_compare_xgb[df_compare_xgb['dif']==0])
accuracyy = ddd/len(df_compare_xgb)*100
print('xgb forest accuracy for last '+str(shift)+' '+ticker+' '+str(accuracyy))
msft = yf.Ticker(ticker.upper())
df = msft.history(period="max")
ddd = df['Open'].iloc[int(len(df))-days_back-1:]
df__ = {'open':[]}
for x in range(len(ddd)):
  df__['open'].append(int(ddd[x]))
df_final_xgb = pd.DataFrame()
df_final_xgb[str(ticker.upper())+' prediction'] = df__['open'] + df_compare_xgb['prediction']/100*df__['open']
x11 = df_final_xgb[str(ticker.upper())+' prediction']


xt = ttt.head(days_back+1)
yt = yt.reset_index().drop(columns = 'index')
df_compare_gb = pd.DataFrame()
df_compare_gb['prediction'] = list(gb_clf.predict(xt))
yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_gb.mean()[-1:])
yt = yt.reset_index().drop(columns = 'index')[0]
df_compare_gb['actual'] = yt
df_compare_gb['dif'] = abs(df_compare_gb['actual']-df_compare_gb['prediction'])
dddd = len(df_compare_gb[df_compare_gb['dif']==0])
accuracyyy = dddd/len(df_compare_gb)*100
print('gb forest accuracy for last '+str(shift)+' '+ticker+' '+str(accuracyyy))
msft = yf.Ticker(ticker.upper())
df = msft.history(period="max")
dddd = df['Open'].iloc[int(len(df))-days_back-1:]
df___ = {'open':[]}
for x in range(len(dddd)):
  df___['open'].append(int(dddd[x]))
df_finalll = pd.DataFrame()
df_finalll[str(ticker.upper())+' prediction'] = df___['open'] + df_compare_gb['prediction']/100*df___['open']
x111 = df_finalll[str(ticker.upper())+' prediction']

df_check = pd.DataFrame()
df_check['rf'] = y1
df_check['gb'] = x111
df_check['xgb'] = x11
df_check['actual'] = dd.reset_index().drop(columns = ['Date']).head(days_back)

plt.figure(figsize = (20, 10))
ax = y1.head(days_back).plot(label='rf prediction')
ax = (df_['open'] + df_compare_rf['actual']/100*df_['open']).plot(label='Actual')
ax = x11.head(days_back).plot(label='xgb prediction')
ax = x111.head(days_back).plot(label='gb prediction')
ax.set_xlabel('Date')
ax.set_ylabel(str(ticker.upper()))
plt.axvline(x = days_back-1, color = 'b', label = 'axvline - full height')

leg = plt.legend()
for line in leg.get_lines():
    line.set_linewidth(10)
for text in leg.get_texts():
    text.set_fontsize('x-large')

plt.legend()
plt.show()

df_check

default (on/off)
       * specs: up to 3 categories, you choose ticker+forecast period, 2k trees, trained on last 50 days of each stock's returns!
on
input ticker
coin
model will predict xx days into the future
0
run a random forest: (yes/no)
yes
run a xgboost (yes/no):
yes
run a logit (yes/no):
no
run a gradient boosted (yes/no):
yes
enter your industry categories, according to the number you referrenced above
tech
software
internet
MCHP
ANSS
PATH
FICO
NEM.F
AZPN
HCP
AVV.L
SGE.L
4716.T
IOT
FIVN
SMAR
IBKR
377300.KS
DHER.F
BILI
W


In [23]:
import os
import tweepy as tw
import pandas as pd

time.sleep(60)
consumer_key= '7d7PKS74mTCa5EYGc6lw5KHyH'
consumer_secret= 'FG2d31TVlPaR7zTedvsQUX9vkC5c4tInorAOPIdJ0nMNT2GqmP'
access_token= '1343573569796767745-TDUJmh3IEyPIY390WkW1N10b9uBPZG'
access_token_secret= '1PQ1MuLQtm82v6ML9ks908J75xvppFFkn1kH3cd4yLTyA'


auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


search_words = '#$TSLA'
search = search_words + " -filter:retweets"


tweets = tw.Cursor(api.search,
                    q=search,
                    lang="en",
                    since='2022-01-01').items(1000)


all_tweets = [[tweet.text, tweet.id, str(tweet.created_at.date()),tweet.user.location,tweet.entities] for tweet in tweets]

df = pd.DataFrame(all_tweets)

In [ ]:
def option_pricing(ticker,num_categories ,default, days_back,descriptive,scrape,num_trees,setup,shift_change,current_price,div,r,shift):
  #------------------------------------------------------
  if scrape == 'yes':
    urls = [
      'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
      'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/',
      'https://companiesmarketcap.com/airlines/largest-airlines-by-market-cap/',
      'https://companiesmarketcap.com/airports/largest-airport-operating-companies-by-market-cap/',
      'https://companiesmarketcap.com/aircraft-manufacturers/largest-aircraft-manufacturers-by-market-cap/',
      'https://companiesmarketcap.com/banks/largest-banks-by-market-cap/',
      'https://companiesmarketcap.com/hotels/largest-hotel-companies-by-market-cap/',
      'https://companiesmarketcap.com/pharmaceuticals/largest-pharmaceutical-companies-by-market-cap/',
      'https://companiesmarketcap.com/e-commerce/largest-e-commerce-companies-by-market-cap/',
      'https://companiesmarketcap.com/healthcare/largest-healthcare-companies-by-market-cap/',
      'https://companiesmarketcap.com/ports/largest-port-operating-companies-by-market-cap/',
      'https://companiesmarketcap.com/professional-services/largest-professional-service-companies-by-market-cap/',
      'https://companiesmarketcap.com/food/largest-food-companies-by-market-cap/',
      'https://companiesmarketcap.com/restaurant-chains/largest-restaurant-chain-companies-by-market-cap/',
      'https://companiesmarketcap.com/software/largest-software-companies-by-market-cap/',
      'https://companiesmarketcap.com/semiconductors/largest-semiconductor-companies-by-market-cap/',
      'https://companiesmarketcap.com/tobacco/largest-tobacco-companies-by-market-cap/',
      'https://companiesmarketcap.com/financial-services/largest-financial-service-companies-by-market-cap/',
      'https://companiesmarketcap.com/oil-gas/largest-oil-and-gas-companies-by-market-cap/',
      'https://companiesmarketcap.com/electricity/largest-electricity-companies-by-market-cap/',
      'https://companiesmarketcap.com/delivery-services/largest-delivery-companies-by-market-cap/',
      'https://companiesmarketcap.com/media-press/largest-media-and-press-companies-by-market-cap/',
      'https://companiesmarketcap.com/alcoholic-beverages/largest-alcoholic-beverage-companies-by-market-cap/',
      'https://companiesmarketcap.com/beverages/largest-beverage-companies-by-market-cap/',
      'https://companiesmarketcap.com/clothing/largest-clothing-companies-by-market-cap/',
      'https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap/',
      'https://companiesmarketcap.com/railways/largest-railways-companies-by-market-cap/',
      'https://companiesmarketcap.com/insurance/largest-insurance-companies-by-market-cap/',
      'https://companiesmarketcap.com/real-estate/largest-real-estate-companies-by-market-cap/',
      'https://companiesmarketcap.com/chemicals/largest-chemical-companies-by-market-cap/',
      'https://companiesmarketcap.com/investment/largest-investment-companies-by-market-cap/',
      'https://companiesmarketcap.com/telecommunication/largest-telecommunication-companies-by-market-cap/',
      'https://companiesmarketcap.com/retail/largest-retail-companies-by-market-cap/',
      'https://companiesmarketcap.com/internet/largest-internet-companies-by-market-cap/',
      'https://companiesmarketcap.com/construction/largest-construction-companies-by-market-cap/',
      'https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/'
    ]
    a = []
    for x in range(len(urls)):
        a.append(urls[x].split('/')[3])

    import requests
    import time
    import bs4
    from bs4 import BeautifulSoup
    cats = ['automakers','airlines','aircraft-manufacturers','banks','pharmaceuticals','e-commerce','healthcare','ports','professional-services','food','restaurant-chains','software','semiconductors','tobacco','financial-services','electricity','courier-services','media/press','alcoholic-beverages','beverages','clothing','mining','railways','insurance','real-estate','chemicals','investment','telecomunication','retail','internet','construction','tech']
    cat_names = {'category':[],'names':[],'ticker':[]}
    try:
        for x in range(len(urls)):
            url = urls[x]
            html = requests.get(url).text
            soup = bs4.BeautifulSoup(html, "html.parser")
            for tag in soup.findAll("div", {"class": "name-div"}):
                    cat_names['names'].append(tag.findNext("div").text)
                    cat_names['category'].append(a[x])
            for tag in soup.findAll("div", {"class": "company-name"}):
                    cat_names['ticker'].append(tag.findNext().text)

    except:
        print(x)
  else:
    print('no scrape')

  if setup == 'on':
    cat_names = pd.DataFrame(cat_names)
    print('enter your industry categories, according to the number you referrenced above')
    gg = []  
    tech = pd.DataFrame()
    cat_holder = []
    for x in range(num_catzz):
      x = xyy['title'][x]
      cat_holder.append(x)
      tech = (cat_names['ticker'][cat_names['category']==x])
      gg.append(tech)
    tech = pd.concat(gg,ignore_index = True)
    tech = tech.drop_duplicates()
      

    for x in range(len(tech)):
      try: #from yfinance module (searching daily returns from tech symbols on marketcap.com)
          xx  = yf.Ticker(tech.iloc[x])
          df = xx.history(period="max")
          df.columns.values[0] = tech.iloc[x]+" Open"
          df.columns.values[1] = tech.iloc[x]+" High"
          df.columns.values[2] = tech.iloc[x]+" Low"
          df.columns.values[3] = tech.iloc[x]+" Close"
          df.columns.values[4] = tech.iloc[x]+" Volume"
          df.columns.values[5] = tech.iloc[x]+" Dividends"
          df.columns.values[6] = tech.iloc[x]+" Stock Splits"
          tech.iloc[x] = df.reset_index()
      except:
        print(tech.iloc[x])

    msft = yf.Ticker(ticker.upper())
    df = msft.history(period="max")
    for ii in range(len(tech)):
      if len(tech.iloc[ii])>=days_back:
        df = pd.merge(df, tech.iloc[ii] ,on = 'Date')
      else:
        print('not long enough!')
    for x in range(num_catzz):
      x = xyy['title'][x]
      cat_holder.append(x)
      tech = (cat_names['ticker'][cat_names['category']==x])
      gg.append(tech)
    tech = pd.concat(gg,ignore_index = True)
    tech = tech.reset_index()
    tech = tech.drop(columns = 'index')
    tech = tech.drop_duplicates()
    tech = tech['ticker']
    aa = pd.DataFrame()
    vol = pd.DataFrame()
    ss = []
    for i in range(len(tech)):
      try:
        aa[str(tech.iloc[i])] = (df[str(tech.iloc[i])+' Close'].shift(-1*shift) - df[str(tech.iloc[i])+' Open'].shift(shift)).dropna()/df[str(tech.iloc[i])+' Open'].shift(shift).dropna()*100
        vol[str(tech.iloc[i])] = df[str(tech.iloc[i])+' Volume']
      except:
        ss.append(str(tech.iloc[i]))
    aa = round(aa)
    tech = pd.DataFrame(tech)
    dd = pd.DataFrame()
    rr = pd.DataFrame()
    for x in range(len(tech)):
      try:
        rr[tech.iloc[x]] = aa[tech.iloc[x]]
      except:
        print(tech.iloc[x])
    qq = pd.DataFrame()
    for x in range(len(tech)):
      try:
        qq[tech.iloc[x]] = aa[tech.iloc[x]]
      except:
        print(tech.iloc[x])
    ss = []
    df = pd.DataFrame()
    for x in range(len(tech)):
      try:
        df = qq.drop(columns = tech.iloc[x])
        ss.append(df)
      except:
        print(tech.iloc[x])
    pieces = []
    piecess = []
    for x in range(len(tech)):
      try:
        pieces.append(ss[x].iloc[-1*days_back:-1]) #for the training set
        piecess.append(ss[x].iloc[-1*days_back:]) #the test set (using the last x iteration to predict the next return)
      except:
        print(tech.iloc[x])
    df_final = pd.concat(pieces,axis = False, ignore_index = True)
    df_finall = pd.concat(piecess,axis= False, ignore_index = True )
  else:
        print('no setup')
  if shift_change == 'on':
    y = []
    for x in range(len(tech)):
      try:
        y.append(qq[tech['ticker'][x]].iloc[-1*days_back:].shift(shift*-1).dropna()) #shifting y back 1 to represent xx day(s) returns from xx day(s) returns 1 unit back
      except:
        print(tech.iloc[x])
    y = pd.concat(y, ignore_index = True)
    yy =y.reset_index()
    yy = yy.drop(columns = 'index')
    yy[0].loc[~(yy[0]==0)]
    yyy = pd.DataFrame()
    yy = yy.dropna()
    yyy['y'] = yy[0]
    yyy = yyy.reset_index()
    tt = round(df_final)
    ttt = round(df_finall)
    tt = tt.fillna(1)
    ttt = ttt.fillna(1)
    xx = tt.reset_index()

    xx = xx.drop(columns = 'index')
    xx = xx.reset_index()
    xxx = pd.merge(yyy,xx, on = 'index')
    y = xxx['y']
    xxx = xxx.drop(columns = ['index','y'])
    def clean_dataset(df):
      assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
      df.dropna(inplace=True)
      indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
      return df[indices_to_keep].astype(np.float64)
    xxx = clean_dataset(xxx).reset_index()
    yy = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index')
    yy = yy['y'].astype(int)
    xxx = pd.merge(y.reset_index(),clean_dataset(xxx).reset_index(), on = 'index').drop(columns = ['y','index','level_0']).astype(int)
    x = xxx
    xx = xxx
    xxx = xxx
    y = yy
    yy = yy
    yyy = yy
  else:
    print('no setup')
  if random_forest == 'yes':
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    clf=RandomForestClassifier(n_estimators=num_trees)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print('random forest score = '+str(metrics.accuracy_score(y_test, y_pred)))
  else:
    print('no random forest')
  if logi_regression == 'yes':
    xx, yy = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)

    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
  else:
    print('no logistic regression')
  if gradient_boosted == 'yes':
    X_train, X_test, y_train, y_test = train_test_split(xxx, yyy, test_size=0.2)
    lr_list = [0.05]
    for learning_rate in lr_list:
        gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
        gb_clf.fit(X_train, y_train)
        print("Learning rate: ", learning_rate)
        print("Gradient Boosted Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
        print("Gradient Boosted Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))
  else:
    print('no gradient boosted forests')
  if xgb == 'yes':
    xgb_clf = XGBClassifier()
    xgb_clf.fit(X_train, y_train)
    score = xgb_clf.score(X_test, y_test)
    print('XGB Score = '+str(score))
    xt = x.head(days_back)
    yt = x.iloc[days_back:days_back][ticker.upper()]
    yt = yt.reset_index().drop(columns = 'index')
    df_compare_xgb = pd.DataFrame()
    df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
    df_compare_xgb['actual'] = yt
    df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
    dd = len(df_compare_xgb[df_compare_xgb['dif']==0])
    accuracy = dd/len(df_compare_xgb)*100
    print('xgb forest accuracy for last '+str(shift)+' '+ticker+' '+str(accuracy))
  else:
    print('no xg boosted forest')

  #testing + graphing
  ttt = ttt
  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_rf = pd.DataFrame()
  df_compare_rf['prediction'] = list(clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_rf.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  yt
  df_compare_rf['actual'] = yt
  df_compare_rf['dif'] = abs(df_compare_rf['actual']-df_compare_rf['prediction'])
  dd = len(df_compare_rf[df_compare_rf['dif']==0])
  accuracy = dd/len(df_compare_rf)*100
  print('random forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  dd = df['Open'].iloc[int(len(df))-days_back-1:]
  df_ = {'open':[]}
  for x in range(len(dd)):
    df_['open'].append(int(dd[x]))
  df_final = pd.DataFrame()
  df_final[str(ticker.upper())+' prediction'] = df_['open'] + df_compare_rf['prediction']/100*df_['open']
  y1 = df_final[str(ticker.upper())+' prediction']


  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_xgb = pd.DataFrame()
  df_compare_xgb['prediction'] = list(xgb_clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_xgb.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  df_compare_xgb['actual'] = yt
  df_compare_xgb['dif'] = abs(df_compare_xgb['actual']-df_compare_xgb['prediction'])
  ddd = len(df_compare_xgb[df_compare_xgb['dif']==0])
  accuracyy = ddd/len(df_compare_xgb)*100
  print('xgb forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracyy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  ddd = df['Open'].iloc[int(len(df))-days_back-1:]
  df__ = {'open':[]}
  for x in range(len(ddd)):
    df__['open'].append(int(ddd[x]))
  df_final_xgb = pd.DataFrame()
  df_final_xgb[str(ticker.upper())+' prediction'] = df__['open'] + df_compare_xgb['prediction']/100*df__['open']
  x11 = df_final_xgb[str(ticker.upper())+' prediction']


  xt = ttt.head(days_back+1)
  yt = yt.reset_index().drop(columns = 'index')
  df_compare_gb = pd.DataFrame()
  df_compare_gb['prediction'] = list(gb_clf.predict(xt))
  yt = xxx.iloc[days_back:days_back*2][ticker.upper()].append(df_compare_gb.mean()[-1:])
  yt = yt.reset_index().drop(columns = 'index')[0]
  df_compare_gb['actual'] = yt
  df_compare_gb['dif'] = abs(df_compare_gb['actual']-df_compare_gb['prediction'])
  dddd = len(df_compare_gb[df_compare_gb['dif']==0])
  accuracyyy = dddd/len(df_compare_gb)*100
  print('gb forest accuracy for predictions of '+str(shift)+'-day returns of '+ticker+' stock: '+str(accuracyyy))
  msft = yf.Ticker(ticker.upper())
  df = msft.history(period="max")
  dddd = df['Open'].iloc[int(len(df))-days_back-1:]
  df___ = {'open':[]}
  for x in range(len(dddd)):
    df___['open'].append(int(dddd[x]))
  df_finalll = pd.DataFrame()
  df_finalll[str(ticker.upper())+' prediction'] = df___['open'] + df_compare_gb['prediction']/100*df___['open']
  x111 = df_finalll[str(ticker.upper())+' prediction']

  df_check = pd.DataFrame()
  df_check['rf'] = y1
  df_check['gb'] = x111
  df_check['xgb'] = x11
  df_check['actual'] = dd.reset_index().drop(columns = ['Date']).head(days_back)

  plt.figure(figsize = (20, 10))
  ax = y1.head(days_back).plot(label='rf prediction')
  ax = (df_['open'] + df_compare_rf['actual']/100*df_['open']).plot(label='Actual')
  ax = x11.head(days_back).plot(label='xgb prediction')
  ax = x111.head(days_back).plot(label='gb prediction')
  ax.set_xlabel('Date')
  ax.set_ylabel(str(ticker.upper()))
  plt.axvline(x = days_back-1, color = 'b', label = 'axvline - full height')

  leg = plt.legend()
  for line in leg.get_lines():
      line.set_linewidth(10)
  for text in leg.get_texts():
      text.set_fontsize('x-large')

  plt.legend()
  plt.show()
  plt.savefig('predictions plot for '+str(ticker.upper())+'.png')

  print(df_check)

  def options_chain(symbol):
      msft = yf.Ticker(symbol)
      exps = msft.options
      ops = pd.DataFrame()
      for e in exps:
          opt = msft.option_chain(e)
          opt = pd.DataFrame().append(opt.calls).append(opt.puts)
          opt['expirationDate'] = e
          ops = ops.append(opt, ignore_index=True)
      ops['expirationDate'] = pd.to_datetime(ops['expirationDate']) + timedelta(days = 1)
      ops['dte'] = (ops['expirationDate'] - datetime.today()).dt.days
      ops['CALL'] = ops['contractSymbol'].str[0:].apply(
          lambda x: "C" in x)
      ops[['bid', 'ask', 'strike']] = ops[['bid', 'ask', 'strike']].apply(pd.to_numeric)
      ops['mark'] = (ops['bid'] + ops['ask']) / 2*1000
      return ops

  df = options_chain(str(ticker.upper())).sort_values('dte',ascending=True)

  calls = df[df['CALL']==True]
  puts = df[df['CALL']==False]
  puts = puts.sort_values('expirationDate',ascending = True)
  calls = calls.sort_values('expirationDate',ascending = True)

  #black scholes formula
  def d1(S,K,T,r,div,sigma):
      return(log(S/K)+(r-div+sigma**2/2.)*T)/(sigma*sqrt(T))
  def d2(S,K,T,r,div,sigma):
      return d1(S,K,T,r,div,sigma)-sigma*sqrt(T)
  def bs_call(S,K,T,r,div,sigma):
      return S*norm.cdf(d1(S,K,T,r,div,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,div,sigma))
  def bs_put(S,K,T,r,div,sigma):
      return K*exp(-r*T)-S+bs_call(S,K,T,r,div,sigma)
  print('puts:')
  S = current_price
  S1 =  float((df_['open'] + df_compare_rf['actual']/100*df_['open']).drop(columns = 'index')[days_back])
  difference = S1 - S
  pdifference = difference/S

  try:
    Tp = float(puts['dte'].reset_index().drop(columns = 'index')['dte'][xxe])/365
    Kp = float(puts['strike'].reset_index().drop(columns = 'index')['strike'][xxe])
    sigmap = float(puts['impliedVolatility'].reset_index().drop(columns = 'index')['impliedVolatility'][xxe])
    
    #FORMULA: C = price*N( [ln(price/strike)+interest_rate-divYield+vol^2/2)*T]/(vol*sqrt(T)) ] - strike*e^(interest rate * time)*N(d1 - vol*sqrt(T))
    print('Black Scholes Results...')
    zvvv = str(float((puts['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+puts['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('Price of put: '+zvvv)
    zvv = str(Kp-current_price-float((puts['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+puts['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('"immdiate excercise value" of put:'+zvv)
    zv = str(bs_put(S,Kp,Tp,r,div,sigmap))
    print('Black-Scholes expected value of put(from stock open today): '+zv)
    zzv = str(Tp)[:4]+' day(s): '+str(bs_put(S1,Kp,Tp,r,div,sigmap))
    print('Value of put according to avg of ensemble predictsions in '+zzv)
    print(' ---------- ')
  except:
    print('no puts for '+str(ticker.upper()))
  print(' ---------- ')
  print('calls:')
  try:
    Tc = float(calls['dte'].reset_index().drop(columns = 'index')['dte'][xxe])/365
    Kc = float(calls['strike'].reset_index().drop(columns = 'index')['strike'][xxe])
    sigmac = float(calls['impliedVolatility'].reset_index().drop(columns = 'index')['impliedVolatility'][xxe])
    cvvv = str(float((calls['bid'].reset_index().drop(columns = 'index')['bid'][x])+calls['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('Price of call: '+cvvv)
    cvv = str(current_price-Kc-float((calls['bid'].reset_index().drop(columns = 'index')['bid'][xxe])+calls['ask'].reset_index().drop(columns = 'index')['ask'][xxe])/2)
    print('"immdiate excercise value" of call:'+cvv)
    cv = str(bs_call(S,Kc,Tc,r,div,sigmac))
    print('Black-Scholes expected value of call(from stock open today): '+cv)
    ccv = str(Tc)[:]+' day(s): '+str(bs_call(S1,Kc,Tc,r,div,sigmac))
    print('Value of call according to avg of ensemble predictsions in '+ccv)
  except:
    print('no calls for '+str(ticker.upper()))
  
  return ticker, cvvv, cvv, cv,ccv, zvvv, zvv, zv,zzv, S1, difference, pdifference
       # ticker, price of call, immediate excercise value call, BS value, BS pred value, same for put, PRED_PRICE, PRED DIF, PRED P%
xcer =0